In [1]:
import ROOT
import numpy as np

In [2]:
import matplotlib.pyplot as plt
from itertools import accumulate

In [22]:
#mu = ROOT.TFile.Open("mu+.root")
#mcp = ROOT.TFile.Open("mcp+.root")
mu = ROOT.TFile.Open("../build/MCP.root")

# Get the TTrees
mu_ntuple = mu.Get("ntuple")  
#mcp_ntuple = mcp.Get("ntuple")

In [4]:
br_mu = [branch.GetName() for branch in mu_ntuple.GetListOfBranches()]
print(br_mu)


['evt', 'x', 'y', 'z', 't', 'dedx', 'theta', 'dx', 'dy', 'dz', 'momentum']


In [23]:
df = ROOT.RDataFrame("ntuple", "../build/MCP.root")
data = df.AsNumpy(["evt","x","dedx","momentum"])  


In [24]:
x = data["x"]
momentum = data["momentum"]
#z = data["z"]
evt = data["evt"]
dedx = data["dedx"]

In [25]:
residual =[]
#x_mu=[]
#en_mu = []
for i in range(1000):
    #en_i=en[evt == i]
    x_i =x[evt == i]
    #en_i =en_i[:-1]
    #x_i = x_i[:-1]
    s=0
    #for j in range(len(en_i)):
#   #     a = a + j
    #    en_mu.append(en_i[j])
    #    x_mu.append(x_i[j])
    for k in x_i:
        s = k + s
        residual.append(s)

In [112]:
a=[]
for i in range(10000):
    en_i = en[evt == i]
    a.append(sum(en_i))

In [116]:
hist = ROOT.TH1F("mu", "Muon;En(MeV);Counts", 100, 725, 950)

# Fill the histogram
for value in a:
    hist.Fill(value)

# Optional: style
hist.SetLineColor(ROOT.kBlue)
hist.SetMarkerStyle(20)

# Fit with Gaussian: "gaus" is a predefined fit function in ROOT
fit_result = hist.Fit("gaus", "S")  # "S" returns a fit result object

# Draw
canvas = ROOT.TCanvas("canvas", "Canvas", 800, 600)
hist.Draw("E")  # "E" for error bars
canvas.Update()
canvas.SaveAs("En_mu.pdf")

****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      466.806
NDf                       =           77
Edm                       =  1.21331e-08
NCalls                    =           61
Constant                  =       398.35   +/-   5.09862     
Mean                      =      852.409   +/-   0.257104    
Sigma                     =      21.4951   +/-   0.165959     	 (limited)


Warning in <TROOT::Append>: Replacing existing TH1: mu (Potential memory leak).
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Info in <TCanvas::Print>: pdf file En_mu.pdf has been created


In [117]:
mcp = ROOT.RDataFrame("ntuple", "mcp+.root")
mcp_data = mcp.AsNumpy(["evt","dx", "energy"])  
x = mcp_data["dx"]
evt = mcp_data["evt"]
en = mcp_data["energy"]
a=[]
for i in range(10000):
    en_i = en[evt == i]
    a.append(sum(en_i))

In [118]:
hist = ROOT.TH1F("mcp", "MCP;En(MeV);Counts", 100, 725, 950)

# Fill the histogram
for value in a:
    hist.Fill(value)

# Optional: style
hist.SetLineColor(ROOT.kBlue)
hist.SetMarkerStyle(20)

# Fit with Gaussian: "gaus" is a predefined fit function in ROOT
fit_result = hist.Fit("gaus", "S")  # "S" returns a fit result object

# Draw
canvas = ROOT.TCanvas("canvas", "Canvas", 800, 600)
hist.Draw("E")  # "E" for error bars
canvas.Update()
canvas.SaveAs("En_mcp.pdf")

****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      376.901
NDf                       =           73
Edm                       =  6.03283e-06
NCalls                    =           53
Constant                  =      395.829   +/-   5.06779     
Mean                      =      852.252   +/-   0.256507    
Sigma                     =      21.8407   +/-   0.170345     	 (limited)


Warning in <TROOT::Append>: Replacing existing TH1: mcp (Potential memory leak).
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Info in <TCanvas::Print>: pdf file En_mcp.pdf has been created


In [26]:
resid_len = np.array(residual)
en_muon = np.array(dedx)
momentum = np.array(momentum)
#en_sel = np.array(en_left)#muon energy
#step_len=x[evt < 1]
#energy =en[evt<1]
#print(len(resid_len))
#print(len(step_len))
#print(len(energy))

In [32]:
# Assume Dr and De are your data arrays
binY = np.logspace(0,2.5,300)
binX = np.logspace(-2,0,300)
#hist2d = ROOT.TH2F("mu", "dE/dr vs r; r (cm); dE/dr (MeV/cm)", 300, 0, 1, len(binY)-1, binY)
hist2d = ROOT.TH2F("mcp", "dE/dr vs r; Momentum (GeV/cm); dE/dr (MeV/cm)", len(binX)-1, binX, 300, 0,10)

# Fill histogram
for xi, yi in zip(momentum, dedx):
    hist2d.Fill(xi, yi)

# Make profile (s dE/dr per r-bin)
profile = hist2d.ProfileX()
profile.SetLineColor(ROOT.kRed)
profile.SetLineWidth(1)
profile.SetMarkerColor(ROOT.kRed)
profile.SetMarkerStyle(1)

# Draw both
canvas = ROOT.TCanvas("c1", "dE/dr vs r", 800, 600)
hist2d.Draw("COLZ")        
profile.Draw("SAME E1")    
#canvas.SetLogx()
canvas.Update()
canvas.SaveAs("mcp.pdf")



Warning in <TFile::Append>: Replacing existing TH1: mcp (Potential memory leak).
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c1
Info in <TCanvas::Print>: pdf file mcp.pdf has been created


In [100]:
mcp = ROOT.RDataFrame("ntuple", "mcp+.root")
mcp_data = mcp.AsNumpy(["evt","dx", "energy"])  
x = mcp_data["dx"]
evt = mcp_data["evt"]
en = mcp_data["energy"]


residual_mcp =[]
x_mcp=[]
en_mcp = []
for i in range(10000):
    en_i = en[evt == i]
    x_i  = x[evt == i]
    en_i = en_i[:-1]
    x_i  = x_i[:-1]
    s=0
    for j in range(len(en_i)):
#        a = a + j
        en_mcp.append(en_i[j])
        x_mcp.append(x_i[j])
    for k in x_i:
        s = k + s
        residual_mcp.append(s) 

In [102]:
#resid_mcp = np.array(residual_mcp)
#en_mcp = np.array(en_mcp)
#x_mcp = np.array(x_mcp)

binY = np.logspace(0,2.5,300)
#hist2d_mcp = ROOT.TH2F("mcp", "dE/dr vs r; r (cm); dE/dr (MeV/cm)", 500, 0, 1.5, 1000, 0, 10)
hist2d_mcp = ROOT.TH2F("mcp", "dE/dr vs r; r (cm); dE/dr (MeV/cm)", 300, 0, 600, len(binY)-1, binY)

for xi, yi in zip(resid_mcp, en_mcp/x_mcp):
    hist2d_mcp.Fill(xi, yi)

# Make profile (s dE/dr per r-bin)
#profile = hist2d.ProfileX()
#profile.SetLineColor(ROOT.kRed)
#profile.SetLineWidth(1)
#profile.SetMarkerColor(ROOT.kRed)
#profile.SetMarkerStyle(1)

# Draw both
canvas = ROOT.TCanvas("c1", "dE/dr vs r", 800, 600)
hist2d_mcp.Draw("COLZ")        
#profile.Draw("SAME E1")    
canvas.SetLogy()
canvas.Update()
canvas.SaveAs("mcp.pdf")

Warning in <TROOT::Append>: Replacing existing TH1: mcp (Potential memory leak).
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c1
Info in <TCanvas::Print>: pdf file mcp.pdf has been created


In [65]:
# Create profiles
profile_mu = hist2d.ProfileX()
profile_mu.SetLineColor(ROOT.kBlue)
profile_mu.SetMarkerColor(ROOT.kBlue)
profile_mu.SetMarkerStyle(2)
profile_mu.SetLineWidth(1)
profile_mu.SetTitle("Average dE/dr vs r; r (cm); dE/dr (MeV/cm)")

profile_mcp = hist2d_mcp.ProfileX()
profile_mcp.SetLineColor(ROOT.kRed)
profile_mcp.SetMarkerColor(ROOT.kRed)
profile_mcp.SetMarkerStyle(1)
profile_mcp.SetLineWidth(1)

# Draw both on the same canvas
c1 = ROOT.TCanvas("c1", "dE/dr vs r", 800, 600)
profile_mu.Draw("E1")           # Base profile (draws axis)
profile_mcp.Draw("SAME E1")     # Overlay with error bars

# Add legend
legend = ROOT.TLegend(0.8, 0.8, 0.9, 0.9)
legend.AddEntry(profile_mu, "Muon")
legend.AddEntry(profile_mcp, "MCP")
legend.Draw()

c1.Update()
c1.SaveAs("muvsmcp.png")


Warning in <TCanvas::Constructor>: Deleting canvas with same name: c1
Info in <TCanvas::Print>: png file muvsmcp.png has been created
